# BEE 4750 Homework 5: Solid Waste Disposal

**Name**: Akshara Chandrabalan

**ID**: ac2837

> **Due Date**
>
> Friday, 11/10/23, 9:00pm

## Overview

### Instructions - TYPO?

-   In Problem 1, you will formulate, solve, and analyze a standard
    generating capacity expansion problem.
-   In Problem 2, you will add a CO<sub>2</sub> constraint to the
    capacity expansion problem and identify changes in the resulting
    solution.

### Load Environment

The following code loads the environment and makes sure all needed
packages are installed. This should be at the start of most Julia
scripts.

``` julia
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
```

``` julia
using JuMP
using HiGHS
using DataFrames
using GraphRecipes
using Plots
using Measures
using MarkdownTables
```

In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

  Activating project at `~/Documents/BEE 5750 /hw05-akshara-c`


In [2]:
using JuMP
using HiGHS
using DataFrames
using GraphRecipes
using Plots
using Measures
using MarkdownTables

## Background

Three cities are developing a coordinated municipal solid waste (MSW)
disposal plan. Three disposal alternatives are being considered: a
landfill (LF), a materials recycling facility (MRF), and a
waste-to-energy facility (WTE). The capacities of these facilities and
the fees for operation and disposal are provided in the table below.

|    **Disposal Facility**     | **Capacity** (Mg/d) | **Fixed cost** (\$/d) | **Tipping Fee ** (\$/Mg) | **Recycling Cost** (\$/Mg) |
|:-------------:|:-------------:|:-------------:|:----------:|:--------------:|
|           Landfill           |         200         |         2000          |            50            |                            |
| Materials Recycling Facility |         350         |         1500          |            7             |    40 (per Mg recycled)    |
|   Waste-to-Energy Facility   |         210         |         2500          |            60            |                            |

Transportation costs are \$1.5/Mg-km, and the relative distances between
the cities and facilities are provided in the table below.

| **City/Facility** | **Landfill (km)** | **MRF (km)** | **WTE (km)** |
|:-----------------:|:-----------------:|:------------:|:------------:|
|         1         |         5         |      30      |      15      |
|         2         |        15         |      25      |      10      |
|         3         |        13         |      45      |      20      |
|        LF         |        \-         |      32      |      18      |
|        MRF        |        32         |      \-      |      15      |
|        WTE        |        18         |      15      |      \-      |

The fixed costs associated with the disposal options are incurred only
if the particular disposal option is implemented. The three cities
produce 100, 90, and 120 Mg/day of solid waste, respectively, with the
composition provided in the table below.

|   **Component**   | **% of total mass** | **Combustion ash** (%) | **MRF Recycling rate** (%) |
|:---------------------:|:--------------:|:---------------:|:---------------:|
|    Food Wastes    |         15          |           8            |             0              |
| Paper & Cardboard |         40          |           7            |             55             |
|     Plastics      |          5          |           5            |             15             |
|     Textiles      |          3          |           10           |             10             |
|  Rubber, Leather  |          2          |           15           |             0              |
|       Wood        |          5          |           2            |             30             |
|    Yard Wastes    |         18          |           2            |             40             |
|       Glass       |          4          |          100           |             60             |
|      Ferrous      |          2          |          100           |             75             |
|     Aluminum      |          2          |          100           |             80             |
|    Other Metal    |          1          |          100           |             50             |
|   Miscellaneous   |          3          |           70           |             0              |

The information in the above table will help you determine the overall
recycling and ash fractions. Note that the recycling residuals, which
may be sent to either landfill or the WTE, have different ash content
than the ash content of the original MSW. You will need to determine
these fractions to construct your mass balance constraints.

**Reminder**: Use `round(x; digits=n)` to report values to the
appropriate precision!

## Problems (Total: 40 Points)

### Problem 1 (22 points)

In this problem, you will develop an optimal disposal plan for the two
cities.

#### Problem 1.1 (3 points) - DONE

Based on the information above, calculate the overall recycling and ash
fractions for the waste produced by each city.

ANSWER:
$\\$ 
overall recycling fraction = (0.15 * 0) + (0.4 * 0.55) + (0.05 * 0.15) + (0.03 * 0.1) + (0.02 * 0) + (0.05 * 0.3) + (0.18 * 0.4) + (0.04 * 0.6) + (0.02 * 0.75) + (0.02 * 0.8) + (0.01 * 0.5) + (0.03 * 0)

overall recycling fraction = 0.38 (38%)

overall ash fraction = (0.15 * 0.08) + (0.4 * 0.07) + (0.05 * 0.05) + (0.03 * 0.1) + (0.02 * 0.15) + (0.05 * 0.02) + (0.18 * 0.02) + (0.04 * 1) + (0.02 * 1) + (0.02 * 1) + (0.01 * 1) + (0.03 * 0.7)

overall ash fraction = 0.16 (16%)

#### Problem 1.2 (2 points) - DONE

What are the decision variables for your optimization problem? Provide
notation and variable meaning.

ANSWER:
$\\$
$W_{ij}$, in Mg/day, is the amount of waste transported from each city $i \in I$ to a given disposal site $j \in J$ where $I$ = [1:3] and $J$ = [1:3] (J[1] corresponds to LF, J[2] corresponds to MRF and J[3] corresponds to WTE)
$\\$

$R_{kj}$, in Mg/day, is the amount of residual waste transported from disposal $k \in K$ to disposal $j$, where K = [2:3]
$\\$

$Y_j$ is the operational (on/off) status of disposal j - binary and unitless


#### Problem 1.3 (3 points) - DONE

Formulate the objective function. Make sure to include any needed
derivations or justifications for your equation(s).

ANSWER: The objective is to minimize total cost, in dollars. Total cost is made up of transportation costs plus disposal costs.

Transportation costs:
$$\\$$
$a_{ij}$ is the cost of transporting waste from source i to disposal j in dollars/Mg-km
$$\\$$
$l_{ij}$ is the distance between source i and disposal j in km
$$\\$$
Therefore,
$$
transportation \ costs = \sum_{i \in I, j \in J, k \in K} a_{ij} l_{ij} W_{ij} R_{kj} 
$$

Disposal costs:
$$\\$$
$c_j$ is the fixed cost of operating disposal j in dollars/day 
$$\\$$
$b_j$ is the tipping fee (variable cost) in dolars/Mg
$$\\$$
Therefore,
$$
disposal \ cost = \sum_{j \in J} [c_j + b_j[\sum_{i \in I} W_{ij} + \sum_{k \in K} R_{kj}]]
$$

So, the objective function is as follows:
$$
\min_{W_{ij}, Y_{j}, R_{kj}} \sum_{i \in I, j \in J, k \in K} a_{ij} l_{ij} W_{ij} R_{kj} + \sum_{j \in J} [c_jY_j + b_j\sum_{i \in I, k \in K} [W_{ij} + R_{kj}]]
$$

Process of simplifying:
$\\$
$LF cost = 2000Y_1 + 50(W_{11} + W_{21} + W_{31} + R_{21} + R_{31})$ 
$\\$
$MRF cost = 1500Y_2 + 7(W_{12} + W_{22} + W_{32}) + 0.38(40)(W_{12} + W_{22} + W_{32})$ 
$\\$
$WTE cost = 2500Y_3 + 60(W_{13} + W_{23} + W_{33} + R_{23})$
$\\$
$total \ transportation \ cost = 1.5[5W_{11} + 15W_{21} + 13W_{31} + 32R_{21} + 18R_{31} + 30W_{12} + 25W_{22} + 45W_{32} + 15W_{13} + 10W_{23} + 20W_{33} + 15R_{23}]$

$$
\min_{W_{ij}, Y_{j}, R_{kj}} 2000Y_1 + 50W_{11} + 50W_{21} + 50W_{31} + 50R_{21} + 50R_{31} + 1500Y_2 + 7W_{12} + 7W_{22} + 7W_{32} + 15.2W_{12} + 15.2W_{22} + 15.2W_{32} + 2500Y_3 + 60W_{13} + 60W_{23} + 60W_{33} + 60R_{23} + 7.5W_{11} + 22.5W_{21} + 19.5W_{31} + 48R_{21} + 27R_{31} + 45W_{12} + 37.5W_{22} + 67.5W_{32} + 22.5W_{13} + 15W_{23} + 30W_{33} + 22.5R_{23}
$$ 

The simplified objective function to minimize cost (in dollars) is:
$$
\min_{W_{ij}, Y_{j}, R_{kj}} 57.5W_{11} + 67.2W_{12} + 82.5W_{13} + 72.5W_{21} + 59.7W_{22} + 75W_{23} + 69.5W_{31} + 89.7W_{32} + 90W_{33} + 98R_{21} + 77R_{31} + 82.5R_{23} + 2000Y_1 + 1500Y_2 + 2500Y_3 
$$ 


#### Problem 1.4 (4 points) - DONE

Derive all relevant constraints. Make sure to include any needed
justifications or derivations.

ANSWER:
$\\$
City mass-balance constraints:
$$W_{11} + W_{12} + W_{13} = 100 \ Mg/d$$
$$W_{21} + W_{22} + W_{23} = 90 \ Mg/d$$
$$W_{31} + W_{32} + W_{33} = 120 \ Mg/d$$

Residual mass-balance constraints:
$$R_{31} = 0.16(W_{13} + W_{23} + W_{33} + R_{23}) \ (for \ WTE) \$$
$$R_{21} + R_{23} = 0.62(W_{12} + W_{22} + W_{32}) \ (for \ MRF) \$$

Disposal limit constraints:
$$W_{11} + W_{21} + W_{31} + R_{21} + R_{31} \leq 200 \ Mg/d \ (for \ LF) \$$ 
$$W_{12} + W_{22} + W_{32} \leq 350 \ Mg/d \ (for \ MRF) \$$
$$W_{13} + W_{23} + W_{33} + R_{23} \leq 210 \ Mg/d \ (for \ WTE) \$$ 

Commitment and non-negativity constraints:
$$\\$$
LF is always on, so,
$$Y_1 = 1$$

MRF and WTE can be turned on and off, so,
$$Y_2 =
\begin{cases}
    0 & \text{if } W_{12} + W_{22} + W_{32} = 0 \\
    1 & \text{else}
\end{cases}
$$
$$Y_3 =
\begin{cases}
    0 & \text{if } W_{13} + W_{23} + W_{33} + R_{23} = 0 \\
    1 & \text{else}
\end{cases}
$$

Amount of waste and residual cannot be negative, so,
$$W_{ij}, R_{kj} \geq 0$$



#### Problem 1.5 (3 points)

Implement your optimization problem in `JuMP`.

#### Problem 1.6 (2 points)

Find the optimal solution. Report the optimal objective value.

#### Problem 1.7 (5 points)

Draw a diagram showing the flows of waste between the cities and the
facilities. Which facilities (if any) will not be used? Does this
solution make sense?

### Problem 2 (18 points)

It is projected that in the near future the state will introduce a
carbon tax that will increase the cost for transportation and for
disposal by incineration. It is estimated that the additional costs will
be:

-   tipping fee for the WTE facility will increase to\$75/Mg; and
-   transportation costs will increase to \$2/Mg-km.

In this context, the cities are considering adding another landfill and
want to know if this would be cost-effective compared to using the
current facilities with the carbon tax. This landfill would have a
maximum capacity of 100 Mg/day and would be located with the following
distances from the existing sites (excluding LF1):

| **City/Facility** | **Distance to LF2 (km)** |
|:-----------------:|:------------------------:|
|         1         |            45            |
|         2         |            35            |
|         3         |            15            |
|        MRF        |            35            |
|        WTE        |            50            |

The fixed cost of operating this facility would be the same as the first
landfill, but the tipping cost would be increased to \$60/Mg-day.

#### Problem 2.1 (5 points)

What changes are needed to your optimization program from Problem 1 for
this decision problem? Formulate any different variables, objectives,
and/or constraints.

#### Problem 2.2 (3 points)

Implement the new optimization problem in `JuMP`.

#### Problem 2.3 (5 points)

Find the optimal solution and report the optimal objective value.
Provide a diagram showing the new waste flows.

#### Problem 2.4 (5 points)

Would you recommend that the cities build the new landfill? Why or why
not? Your answer should be based on your analysis but can draw on other
considerations as appropriate or desired.

## References

List any external references consulted, including classmates.

https://ctan.math.washington.edu/tex-archive/info/undergradmath/undergradmath.pdf